In [129]:
# standard library imports
from __future__ import absolute_import, division, print_function

import keras

# standard numerical library imports
import pandas
import numpy as np
import scipy as sp

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split, remap_pids, to_categorical

import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [130]:
jet_part_sub_all_175 = pandas.read_csv("../output_all_stable_particles_175.csv")
jet_part_sub_all_175["m"]=0.175
jet_part_sub_175 = [y for x,y in jet_part_sub_all_175.groupby(["entry"])]
len(jet_part_sub_175)

10000

In [131]:
jet_part_sub_all_180 = pandas.read_csv("../output_all_stable_particles_180.csv")
jet_part_sub_all_180["m"]= 0.180
jet_part_sub_180 = [y for x,y in jet_part_sub_all_180.groupby(["entry"])]
len(jet_part_sub_180)

10000

In [132]:
jet_size_175 = []
for i in range(len(jet_part_sub_175)):
    jet_size_175.append(len(jet_part_sub_175[i]))

jet_size_180 = []
for i in range(len(jet_part_sub_180)):
    jet_size_180.append(len(jet_part_sub_180[i]))

In [133]:
np.average(jet_size_175), np.average(jet_size_180)

(538.4231, 538.9929)

In [134]:
np.min(jet_size_175), np.min(jet_size_180)

(125, 122)

In [135]:
np.max(jet_size_175), np.max(jet_size_180)

(1113, 1192)

In [136]:
max_jet_size = max(np.max(jet_size_175), np.max(jet_size_180))

We want N X 1200 X 4

In [137]:
Y0 = np.zeros((int(len(jet_part_sub_175)/3)))
Y1 = np.ones((int(len(jet_part_sub_180)/3)))
Y  = np.concatenate([Y0,Y1])
Y = to_categorical(Y, num_classes=2)

In [138]:
feature_to_use = ["pT","eta","phi","pid","m"]
num_feature = len(feature_to_use)

In [139]:
X0_all = np.zeros((len(jet_part_sub_175), max_jet_size,num_feature))
X1_all = np.zeros((len(jet_part_sub_180), max_jet_size,num_feature))

In [140]:
for i in range(len(jet_part_sub_175)):
    jets = np.array(jet_part_sub_175[i][feature_to_use])
    X0_all[i,:len(jets),:] = jets[:max_jet_size]

In [141]:
for i in range(len(jet_part_sub_180)):
    jets = np.array(jet_part_sub_180[i][feature_to_use])
    X1_all[i,:len(jets),:] = jets[:max_jet_size]

In [142]:
remap_pids(X0_all, pid_i=3)
remap_pids(X1_all, pid_i=3)

In [143]:
X0_train = X0_all[:int(len(jet_part_sub_175)/3),:,:]
X1_train = X0_all[:int(len(jet_part_sub_180)/3),:,:]
X0_test = X0_all[int(len(jet_part_sub_175)/2):int(len(2*jet_part_sub_180)/3),:,:]
X1_test = X1_all[int(len(jet_part_sub_180)/2):int(len(2*jet_part_sub_180)/3),:,:]

In [144]:
X0_train.shape, X1_train.shape

((3333, 1192, 5), (3333, 1192, 5))

In [145]:
X = np.concatenate([X0_train,X1_train])

In [146]:
print(np.sum(X[1,:,0]))

735.573565333


In [147]:
for i in range(len(X)):
    X[i] = normalize(X[i])

In [148]:
X_train, X_val, Y_train, Y_val = data_split(X, Y, test=0.3, shuffle=True)

In [149]:
print(X_train.shape)
print(Y_train.shape)

print(X_val.shape)
print(Y_val.shape)

(4667, 1192, 5)
(4667, 2)
(1999, 1192, 5)
(1999, 2)


In [150]:
# network architecture parameters
Phi_sizes = (100,100, 128)
F_sizes = (100,100, 100)

dctr = PFN(input_dim= num_feature, 
           Phi_sizes=Phi_sizes, F_sizes=F_sizes,
           summary=False)

In [151]:
save_label = 'DCTR_top_tagging'

checkpoint = keras.callbacks.ModelCheckpoint(save_label + '.h5', 
                                                monitor='val_loss', 
                                                verbose=2, 
                                                save_best_only=True, 
                                                mode='min')

CSVLogger = keras.callbacks.CSVLogger(save_label + '_loss.csv', append=False)

EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              min_delta=0, 
                                              patience=10, 
                                              verbose=1, 
                                              restore_best_weights=True)

callbacks = [checkpoint, CSVLogger, EarlyStopping]

In [152]:
history = dctr.fit(X_train, Y_train,
                    epochs = 10,
                    batch_size = 1,
                    validation_data = (X_val, Y_val),
                    verbose = 1, 
                    callbacks = callbacks)

Train on 4667 samples, validate on 1999 samples
Epoch 1/10
4667/4667 [==============================] - 111s 24ms/step - loss: 8.0262 - acc: 0.5020 - val_loss: 8.1356 - val_acc: 0.4952

Epoch 00001: val_loss improved from inf to 8.13565, saving model to DCTR_top_tagging.h5
Epoch 2/10
4667/4667 [==============================] - 86s 18ms/step - loss: 8.0262 - acc: 0.5020 - val_loss: 8.1356 - val_acc: 0.4952

Epoch 00002: val_loss did not improve from 8.13565
Epoch 3/10
4667/4667 [==============================] - 64s 14ms/step - loss: 8.0262 - acc: 0.5020 - val_loss: 8.1356 - val_acc: 0.4952

Epoch 00003: val_loss did not improve from 8.13565
Epoch 4/10
4667/4667 [==============================] - 66s 14ms/step - loss: 8.0262 - acc: 0.5020 - val_loss: 8.1356 - val_acc: 0.4952

Epoch 00004: val_loss did not improve from 8.13565
Epoch 5/10
4667/4667 [==============================] - 72s 15ms/step - loss: 8.0262 - acc: 0.5020 - val_loss: 8.1356 - val_acc: 0.4952

Epoch 00005: val_loss did

In [153]:
preds_0 = dctr.predict(X0_test, batch_size=1)
preds_1 = dctr.predict(X1_test, batch_size=1)

In [154]:
preds_0,preds_1

(array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32))

In [155]:
weights_0 = preds_0[:,0]/preds_0[:,1]
weights_1 = preds_1[:,0]/preds_1[:,1]

/root/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/root/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app


In [156]:
print(max(weights_0))
print(max(1/weights_0))
print(max(weights_1))
print(max(1/weights_1))

inf
0.0
inf
0.0


In [157]:
class AddParams2Input(keras.layers.Layer):
    """ Custom layer for tuning with DCTR: 
    Arguments:
    - n_MC_params : (int) - the number of n_MC_params that are in X_dim
    - default_MC_params : (list of floats) - default values for each of the MC parameters
    - trainable_MC_params : (list of booleans) - True for parameters that you want to fit, false for parameters that should be fixed at default value

    Usage: 
    Let X_dim be the input dimension of each particle to a PFN model, and n_MC_params be the number of MC parameters. 
    Defines a Layer that takes in an array of dimension 
    (batch_size, padded_multiplicity, X_dim - n_MC_params)
    This layer appends each particle by the default_MC_params and makes then trainable or non-trainable based on trainable_MC_params
    """
    
    def __init__(self, n_MC_params, default_MC_params, trainable_MC_params):
        super(AddParams2Input, self).__init__()
        # Definitions
        self.n_MC_params = n_MC_params
        self.MC_params = default_MC_params
        self.trainable_MC_params = trainable_MC_params

    
    def build(self, input_shape):
        # Convert input MC parameters to weights and make then trainable or non-trainable
        for i in range(self.n_MC_params):
            self.MC_params[i] = self.add_weight(name='MC_param_{}'.format(i), 
                                                shape=(1, 1),
                                                initializer=keras.initializers.Constant(self.MC_params[i]),
                                                trainable=self.trainable_MC_params[i])
            
        self.MC_params = keras.backend.tf.concat(self.MC_params, axis = -1)
        super(AddParams2Input, self).build(input_shape)
    
    def call(self, input):
        # Add MC params to each input particle (but not to the padded rows)
        concat_input_and_params = keras.backend.tf.where(keras.backend.abs(input[...,0])>0,
                                                         self.MC_params*keras.backend.ones_like(input[...,0:self.n_MC_params]),
                                                         keras.backend.zeros_like(input[...,0:self.n_MC_params]))
        return keras.backend.concatenate([input, concat_input_and_params], -1)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1]+self.n_MC_params)

In [158]:
def get_DCTR_fit_model(DCTR_model, 
                       X_dim, 
                       n_MC_params, 
                       default_MC_params,
                       trainable_MC_params):
    """ 
    Get a DCTR model that trains on the input MC parameters
    
    Arguments:
    - DCTR_model : a PFN model that has been trained on a to continuously interpolate over the input MC dimensions
    - X_dim : (int) - the dimension of the input expected by DCTR_model
    - n_MC_params : (int) - the number of n_MC_params that are in X_dim
    - default_MC_params : (list of floats) - default values for each of the MC parameters
    - trainable_MC_params : (list of booleans) - True for parameters that you want to fit, false for parameters that should be fixed at default value

    Returns:
    - DCTR_fit_model: a compiled model that gradient descends only on the trainable MC parameters
    """
    
    # Do sanity checks on inputs
    assert X_dim >=n_MC_params, "X_dim must be larger than n_MC_params. X_dim includes the dimensionality of the 4-vector + number of MC parameters"
    assert n_MC_params == len(default_MC_params), "Dimension mismatch between n_MC_params and number of default MC parameters given. len(default_MC_params) must equal n_MC_params"
    assert n_MC_params == len(trainable_MC_params), "Dimension mismatch between n_MC_params and trainable_MC_params. len(trainable_MC_params) must equal n_MC_params."
    assert np.any(trainable_MC_params), "All parameters are set to non-trainable."
    
    # Define input to DCTR_fit_model
    non_param_input = keras.layers.Input((None, X_dim - n_MC_params))

    # Construct layer that adds trainable and non-trainable parameters to the input
    add_params_layer = AddParams2Input(n_MC_params, default_MC_params, trainable_MC_params)
    time_dist     = keras.layers.TimeDistributed(add_params_layer, name='tdist')(non_param_input)     

    # Set all weights in DCTR_model to non-trainable
    for layer in DCTR_model.model.layers:
        layer.trainable = False
        
    # get the graph and the weights from the DCTR_model
    output = DCTR_model.model(inputs = time_dist)

    # Define full model
    DCTR_fit_model = fitmodel = keras.models.Model(inputs = non_param_input, outputs = output)
    
    optimizer = keras.optimizers.Adam(lr=1e-4)
    
    # Compile with loss function
    DCTR_fit_model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    
    return DCTR_fit_model

In [159]:
dctr_fit_model = get_DCTR_fit_model(dctr, 
                       X_dim =5, 
                       n_MC_params = 1, 
                       default_MC_params   = [0.175], # default params for [alpha_s, aLund, StoUD]
                       trainable_MC_params = [True]) # Only train aLund

dctr_fit_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, 4)           0         
_________________________________________________________________
tdist (TimeDistributed)      (None, None, 5)           1         
_________________________________________________________________
model_5 (Model)              (None, 2)                 56930     
Total params: 56,931
Trainable params: 1
Non-trainable params: 56,930
_________________________________________________________________


In [160]:
def set_MC_params(dctr_fit_model, MC_params):
    top_mass = MC_params
    weights = np.array([[top_mass]],dtype=np.float32)
    dctr_fit_model.layers[1].set_weights(weights)

In [161]:
dctr_fit_model.layers[1].get_weights()[0].shape

(1, 1)

In [162]:
X.shape, Y.shape

((6666, 1192, 5), (6666, 2))

In [163]:
X0_fit = X0_all[int(2*len(jet_part_sub_175)/3):9999,:,:]
X1_fit = X1_all[int(2*len(jet_part_sub_180)/3):9999,:,:]
X_fit = np.concatenate([X0_fit,X1_fit])[:,:,:4]

In [164]:
Y_fit = Y[:len(X_fit)]

In [165]:
X_fit.shape,Y_fit.shape

((6666, 1192, 4), (6666, 2))

In [166]:
X_fit, _, Y_fit, _ = data_split(X_fit, Y_fit, test=0, shuffle=True)

In [167]:
def get_loss(X, Y, dctr_fit_model, MC_params, batch_size = 1000):
    set_MC_params(dctr_fit_model, MC_params)
    return dctr_fit_model.evaluate(x=X, y = Y, batch_size=batch_size)

In [168]:
dctr_fit_model.layers[1].get_weights()

[array([[0.175]], dtype=float32)]

In [169]:
top_mass_loss = np.array([(top_mass, get_loss(X_fit, Y_fit, dctr_fit_model, [top_mass])) for top_mass in np.linspace(0.170,0.190, 31)])

6666/6666 [==============================] - 37s 6ms/step


In [170]:
print(top_mass_loss)

[[0.17       8.05903825]
 [0.17066667 8.05903825]
 [0.17133333 8.05903825]
 [0.172      8.05903825]
 [0.17266667 8.05903825]
 [0.17333333 8.05903825]
 [0.174      8.05903825]
 [0.17466667 8.05903825]
 [0.17533333 8.05903825]
 [0.176      8.05903825]
 [0.17666667 8.05903825]
 [0.17733333 8.05903825]
 [0.178      8.05903825]
 [0.17866667 8.05903825]
 [0.17933333 8.05903825]
 [0.18       8.05903825]
 [0.18066667 8.05903825]
 [0.18133333 8.05903825]
 [0.182      8.05903825]
 [0.18266667 8.05903825]
 [0.18333333 8.05903825]
 [0.184      8.05903825]
 [0.18466667 8.05903825]
 [0.18533333 8.05903825]
 [0.186      8.05903825]
 [0.18666667 8.05903825]
 [0.18733333 8.05903825]
 [0.188      8.05903825]
 [0.18866667 8.05903825]
 [0.18933333 8.05903825]
 [0.19       8.05903825]]
